#데이콘 대회 참여 
## 이미지분류
## [Private 14th]

### https://dacon.io/competitions/official/235957/overview/description

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 30.7 MB/s 


# 라이브러리 불러온다

In [ ]:
import numpy as np
import pandas as pd
import torch
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import timm
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from google.colab import drive
import random as rand

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #gpu 연결

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/dataset"


train_df = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
train_df.head(10)

,file_name,label
0,001.PNG,9
1,002.PNG,4
2,003.PNG,1
3,004.PNG,1
4,005.PNG,6
5,006.PNG,1
6,007.PNG,5
7,008.PNG,8
8,009.PNG,7
9,010.PNG,7


In [ ]:
img_folder = file_path  +'/' + 'train'
img_set  = os.listdir(img_folder)
print(img_folder)
img_set[0]

/content/drive/MyDrive/dataset/train


'012.PNG'

In [ ]:
img_list = []
for i in range(len(os.listdir(img_folder))):
    img_list.append(img_folder +'/' + os.listdir((os.path.join(img_folder)))[0])

In [ ]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    for i in range(len(os.listdir(data_dir))):
        img_path_list.append(data_dir +'/' + os.listdir((os.path.join(data_dir)))[i])
        img_path_list.sort(key = lambda x : int(x.split('/')[-1].split('.')[0]))
    
    label_list.extend(train_df['label'])
    
    return img_path_list,label_list

def get_test_data(data_dir):
    img_path_list = []
    for i in range(len(os.listdir(data_dir))):
        img_path_list.append(data_dir +'/' + os.listdir((os.path.join(data_dir)))[i])
        img_path_list.sort(key = lambda x : int(x.split('/')[-1].split('.')[0]))

    return img_path_list

In [ ]:
img_folder
img_folder2 = file_path + '/' +'test'

In [ ]:
all_img_path, all_label = get_train_data(img_folder)
test_img_path = get_test_data(img_folder2)

# 데이터셋 정의

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode = True, transforms = None):
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        img_path = self.img_path_list[idx]
        image = cv.imread(img_path)

            
        if self.train_mode and transforms is not None:
            label = self.label_list[idx]
            augmentation = rand.randint(0,8)
            if augmentation < 3: #데이터 증강 사용
                pass
            
            elif augmentation == 3:
                image = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)
            
            elif augmentation == 4:
                image = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)
            
            elif augmentation == 5:
                image = image[::-1].copy()
                
            elif augmentation == 6:
                image = image[:,::-1].copy()
            
            elif augmentation == 7:
                image = image[::-1,::-1,:].copy()
            image = self.transforms(image)
            

            return image, label
        else:
            return self.transforms(image)
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_len = int(len(all_img_path) * 0.8) #실험데이터, 검증데이터 분리 
val_len = int(len(all_img_path) * 0.8)

train_img_path = all_img_path[:train_len]
train_label  = all_label[:train_len]

val_img_path = all_img_path[train_len:]
val_label = all_label[train_len:]

In [ ]:
print(len(val_label))
print(len(train_label))

145
578


In [ ]:
train

In [ ]:
train_trans = transforms.Compose([
        transforms.ToPILImage(), #numpy에서 pil이미지로
        transforms.Resize([128, 128]), #128로 크기조정
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #정규화
])

test_trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128, 128]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
traindataset = Custom_dataset(train_img_path, train_label, True, train_trans)
train_loader = DataLoader(traindataset, batch_size = 16, shuffle = True)

valdataset = Custom_dataset(val_img_path, val_label, True, test_trans)
val_loader = DataLoader(valdataset, batch_size = 16, shuffle = False)

In [ ]:
from torchsummary import summary

# 전이학습(vgg16 사용)

In [ ]:
class Custom_model(nn.Module):
    def __init__(self):
        super(Custom_model, self).__init__()
        self.model = timm.create_model('vgg16', pretrained = True, num_classes = 10) #뒷부분만 10개의 클래스를 분류하는 것으로 변경한다.
        
    def forward(self, x):
        x = self.model(x) 
        return x

In [ ]:
model = Custom_model().to(device)

In [ ]:
summary(model, input_size = (3, 128, 128))

In [ ]:
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
label.dtype

torch.int64

#훈련

In [ ]:
def train(model, dataloader, val_loader, optimizer):
    n = len(dataloader)
    
    for epoch in range(1, 10):
        model.train()
        running_loss = 0 #loss 초기화
        
        for img, label in tqdm(iter(dataloader)):
            img, label = img.to(device), label.to(device) #gpu를 연결해준다.
            optimizer.zero_grad() #가중치 초기화
            
            pred = model(img) #예상
            loss = criterion(pred, label) #
            
            loss.backward() #loss값을 미분해준다.
            optimizer.step() 
            
            running_loss += loss.item()
        print('Epoch : ', epoch, 'Loss : ', running_loss / len(dataloader))
        
        model.eval()
        
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for img, label in tqdm(iter(val_loader)):
                img, label = img.to(device), label.to(device)
                pred = model(img)
                val_loss += criterion(pred, label)
                pred = pred.argmax(dim = 1, keepdim = True)
                correct += pred.eq(label.view_as(pred)).sum().item()
        val_acc = 100 * correct / len(val_loader)
        print('Val loss : ', val_loss / len(val_loader), 'Accuracy : ', val_acc , 'correct : ', correct)
        torch.save(model.state_dict(), '/content/drive/MyDrive/dataset/best_model.pth')    

In [ ]:
train(model, train_loader, val_loader, optim)

In [ ]:
check_point = '/content/drive/MyDrive/dataset/best_model.pth'

check_point = torch.load(check_point)
model = Custom_model().to(device)
model.load_state_dict(check_point)
train(model, train_loader, val_loader, optim)

100%|██████████| 37/37 [00:21<00:00,  1.71it/s]


Epoch :  1 Loss :  0.0005494233227108378


100%|██████████| 10/10 [00:03<00:00,  2.67it/s]


Val loss :  tensor(0.1978, device='cuda:0') Accuracy :  1370.0 correct :  137


100%|██████████| 37/37 [00:22<00:00,  1.63it/s]


...

Val loss :  tensor(0.1517, device='cuda:0') Accuracy :  1380.0 correct :  138


100%|██████████| 37/37 [00:22<00:00,  1.65it/s]


Epoch :  9 Loss :  0.0006857495069640651


100%|██████████| 10/10 [00:03<00:00,  2.64it/s]


Val loss :  tensor(0.1858, device='cuda:0') Accuracy :  1390.0 correct :  139


In [ ]:
def predict(model, test_loader):
  model.eval()
  pred_list = []

  with torch.no_grad():
    for img in tqdm(iter(test_loader)):
      img = img.to(device)

      pred = model(img)
      pred = pred.argmax(dim = 1, keepdim = True).squeeze(1)

      pred_list.extend(pred.tolist())

    return pred_list

In [ ]:
test_dataset = Custom_dataset(test_img_path, None, train_mode = False, transforms = test_trans)
test_loader = DataLoader(test_dataset, batch_size = 8, shuffle = False)

check_point = '/content/drive/MyDrive/dataset/best_model.pth'

check_point = torch.load(check_point)
model = Custom_model().to(device)
model.load_state_dict(check_point)

preds = predict(model, test_loader)


100%|██████████| 25/25 [00:06<00:00,  3.94it/s]


In [ ]:
preds[:5]


[7, 1, 3, 2, 6]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/dataset/sample_submission.csv')
submission['label'] = preds

In [ ]:
submission.to_csv('/content/drive/MyDrive/dataset/first_submission.csv', index = False)

## 전이학습, 데이터 증강을 이용한 결과 99%의 정확도를 얻었다.